In [54]:
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

pd.set_option('display.max_columns', 100)

### Veriler, bir Portekiz bankacılık kurumunun doğrudan pazarlama kampanyaları (telefon görüşmeleri) ile ilgilidir. Sınıflandırma hedefi, müşterinin vadeli bir depozitoya abone olup olmayacağını tahmin etmektir (değişken y).

In [32]:
df = pd.read_csv("/kaggle/input/bank-marketing/bank-additional-full.csv",sep=';')

In [33]:
df.sample(10)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
31608,31,blue-collar,married,basic.9y,no,yes,no,cellular,may,thu,1957,1,999,0,nonexistent,-1.8,92.893,-46.2,1.327,5099.1,no
13573,42,blue-collar,married,basic.4y,no,no,no,cellular,jul,thu,214,1,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no
1160,39,admin.,single,unknown,no,no,no,telephone,may,thu,211,3,999,0,nonexistent,1.1,93.994,-36.4,4.855,5191.0,no
29569,40,technician,single,professional.course,no,yes,yes,cellular,apr,mon,538,1,999,0,nonexistent,-1.8,93.075,-47.1,1.405,5099.1,no
25911,40,admin.,married,high.school,no,yes,no,cellular,nov,wed,131,1,999,0,nonexistent,-0.1,93.200,-42.0,4.120,5195.8,no
12760,56,services,married,basic.4y,unknown,no,no,cellular,jul,tue,160,1,999,0,nonexistent,1.4,93.918,-42.7,4.962,5228.1,no
16275,28,technician,single,high.school,no,no,no,cellular,jul,tue,100,7,999,0,nonexistent,1.4,93.918,-42.7,4.961,5228.1,no
19501,35,technician,single,university.degree,no,yes,yes,cellular,aug,thu,30,1,999,0,nonexistent,1.4,93.444,-36.1,4.968,5228.1,no
20448,32,admin.,divorced,university.degree,no,yes,no,cellular,aug,tue,66,1,999,0,nonexistent,1.4,93.444,-36.1,4.966,5228.1,no
9941,30,blue-collar,single,basic.4y,unknown,no,no,telephone,jun,wed,952,3,999,0,nonexistent,1.4,94.465,-41.8,4.959,5228.1,no


In [34]:
df["pdays"].value_counts()

pdays
999    39673
3        439
6        412
4        118
9         64
2         61
7         60
12        58
10        52
5         46
13        36
11        28
1         26
15        24
14        20
8         18
0         15
16        11
17         8
18         7
22         3
19         3
21         2
25         1
26         1
27         1
20         1
Name: count, dtype: int64

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [36]:
df.isna().sum()

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64

In [37]:
object_variables = df.select_dtypes(include=['object']).columns
object_variables

Index(['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',
       'month', 'day_of_week', 'poutcome', 'y'],
      dtype='object')

In [38]:
lb = LabelEncoder()
df[object_variables] = df[object_variables].apply(lambda column: lb.fit_transform(column))

In [39]:
df.sample(12)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
32895,28,1,1,2,1,1,1,1,6,1,267,1,999,1,0,-1.8,92.893,-46.2,1.299,5099.1,0
18278,35,1,1,2,0,0,0,0,3,4,1259,6,999,0,1,1.4,93.918,-42.7,4.963,5228.1,0
32317,35,0,2,6,0,2,0,0,6,0,662,1,12,1,2,-1.8,92.893,-46.2,1.313,5099.1,1
5616,46,0,0,6,0,2,0,1,6,1,115,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0
8965,34,0,2,6,0,0,0,1,4,2,128,1,999,0,1,1.4,94.465,-41.8,4.866,5228.1,0
882,43,9,2,6,0,2,0,1,6,4,255,2,999,0,1,1.1,93.994,-36.4,4.856,5191.0,0
4408,34,7,1,3,0,0,0,1,6,3,211,1,999,0,1,1.1,93.994,-36.4,4.856,5191.0,0
22061,30,0,1,6,0,0,0,0,1,4,345,2,999,0,1,1.4,93.444,-36.1,4.964,5228.1,0
26134,40,1,2,2,0,1,1,0,7,4,245,4,999,0,1,-0.1,93.200,-42.0,4.120,5195.8,0
34581,28,1,2,2,0,2,0,0,6,2,226,1,999,1,0,-1.8,92.893,-46.2,1.266,5099.1,0


In [40]:
x = df.drop("y",axis=1)
y = df["y"]

In [45]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# XGBClassifier

In [62]:
xgb = XGBClassifier()

xgb.fit(x_train, y_train)
pred = xgb.predict(x_test)

# Model Evaluation
accuracyt = accuracy_score(y_test, pred)
classification_rep = classification_report(y_test, pred)

print("Model: ",xgb)
print("")
print(f'Test Accuracy: {accuracy_test}')
print("")
print(f'Classification Report (Test Set):\n{classification_rep}')

Model:  XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

Test Accuracy: 0.9128429230395727

Classification Report (Test Set):
              precision    recall  f1-score   support

           0       0.94      0.96      0.95      7303
           1       0.64      0.55      0.60       935

